In [39]:
#Imports 
#-----------------------------
# bring in the csv's to work with using pandas dataframe 
#-----------------------------


# pandas info http://pandas.pydata.org/

import numpy as np
import pandas as pd
import csv
isin=pd.read_csv("adjData/isin2.csv")
keys= pd.read_csv("adjData/VermontPop.csv")
adjList= pd.read_csv("adjData/nlist.csv")
keyList= keys[['GEOID']]
# two step join to get only maryland-maryland census tracts 
step= pd.merge(adjList,keyList,how='inner',left_on='NEIGHBOR_TRACTID',right_on='GEOID')
adjSet= pd.merge(step,keyList,how='inner',left_on='SOURCE_TRACTID',right_on='GEOID')
keys=pd.merge(keys,isin,how='inner',left_on='GEOID',right_on='GEOID')


In [ ]:
#PRECOMPUTE NEIGHBORS
#-----------------------------
# precompute number of neighbors so we do not leave any adjacent districts this is to make our lives way easier later 
#-----------------------------

# checked maryland values for no zero values so we can look for orphans right after they are created 

partners={}
for i in range(1,keyList.shape[0]):#
    key = keyList.iloc[i]['GEOID']
    value = adjSet[adjSet.SOURCE_TRACTID == key] # adjacent tracts for census tracts key in NEIGHBOR_TRACTID col 
    partners[key] = value.shape[0] 

In [ ]:
# choose a starting discrict and start working 
# for test purposes we are going to make 1 district 24029950100
# could we precomput the number of districts that are adjcant to a disctric and each time they are added subract one -- much faster and
# we can search easily 

#globals
#feasibility constraint so we can make manual changes 
oeps = np.int64(30000)
eps = np.int64(30000)
targetPop=np.int64(625741) 
#function defs
def findDistrict(adjacent , tracts, pop, det ):
    #Check to see if we are done 
    
    if (pop+det >= targetPop).all():
        return tracts
    # choose next district based on selection method
    
    # get adjacent with most edge 
    v=list(adjacent.values())
    k=list(adjacent.keys())
    # index of tract with most neighbors
    toAdd =k[v.index(max(v))]
    
    
    # once selection is made addtract will handle all of the set updates 
    adjacent,tracts,pop = addTract(adjSetl=adjacent,tract=toAdd,dist=tracts,pop=pop)
    # recursice call
    if  (len ( adjacent ) == 0 ):
        print('error!!')    
        return (tracts,pop) 
    #print (tracts)
    return findDistrict(adjacent,tracts,pop,det)

def addTract (adjSetl,tract,dist,pop):
    print(tract)
    # we need to get the index of a given tract  
    ind=(keys[keys.GEOID == tract])['index'].iloc[0]
    if (keys.iloc[ind]['IsIn'].any() == 0 ):
        #add tract to the district 
        dist.append(tract)
        #remove dist from adjset 
        adjSetl[tract] =-1
        keys.set_value(ind, 13 , 1,takeable=True)
        # update pop
        pop=pop+(keys[keys.GEOID == tract])[['POP100']].iloc[0]  
        #update adj set for new adjacent 
        adjSetl=updateAdjSet(localAdjSet=adjSetl,index=tract)
        # deal with oprhaned districts 
        orphans=isOrphan(tract=tract)
        if (orphans != []):
            #deal with new oprhans we are going to create by adding this new tract 
            #being lazy / greedy and i am just going to add them this might get ~ weird 
            for val in orphans:
                #addTract(adjSetl=adjSetl,tract=val,dist=dist,pop=pop)
                afaf=val
    else: 
        #print ('trying to add already added')
        v=list(adjSetl.values())
        k=list(adjSetl.keys())
        toDel =k[v.index(max(v))]
        del adjSetl[toDel]
        
    return (adjSetl,dist,pop)

def isOrphan (tract):
    global partners 
    orphans=[]
    toRemove = adjSet[adjSet.SOURCE_TRACTID == tract] # adjacent tracts for census tracts key in NEIGHBOR_TRACTID col 
    for i in range(1,toRemove.shape[0]):
        key = toRemove.iloc[i]['NEIGHBOR_TRACTID']
        value=partners[key]
        if (value == 1): 
            orphans.append(key)
        partners[key]=value-1
    return orphans

def updateAdjSet(localAdjSet,index):
    toAdd = adjSet[adjSet.SOURCE_TRACTID == index] # adjacent tracts for census tracts key in NEIGHBOR_TRACTID col 
    for i in range(1,toAdd.shape[0]):#
        key = toAdd.iloc[i]['NEIGHBOR_TRACTID']   
        if  key in localAdjSet:
            if (localAdjSet[key]==-1):
                # do nothing this value is alredy been added to the district 
                b=1
            else:
                localAdjSet[key]=localAdjSet[key]+1
        else:
            localAdjSet[key]=1       
    return localAdjSet




In [49]:
a={};
#get the first District 

startTractId1 = 50001960100
first= [startTractId1]
firstStartPop=(keys[keys.GEOID == startTractId1])[['POP100']].iloc[0]
locAdjSet1=updateAdjSet(localAdjSet=a,index=startTractId1)
first= findDistrict(adjacent=locAdjSet1,tracts=first,pop=firstStartPop,det=0)
print(first)


KeyError: 50001960100.0

In [ ]:
#(keys[keys.GEOID == 50023955800])[['index']].iloc[0]
foo=(keys[keys.GEOID == 50023955800])
a=foo['index']
print(a+9999)

In [50]:
foo=(keys[keys.GEOID == 50001960100])['index'].iloc[0]

print(foo)

0


In [51]:
print(keys)

           GEOID  SUMLEV  STATE  COUNTY   CBSA  CSA                  NAME  \
0    50001960100     140     50       1  99999  999     Census Tract 9601   
1    50001960200     140     50       1  99999  999     Census Tract 9602   
2    50001960300     140     50       1  99999  999     Census Tract 9603   
3    50001960400     140     50       1  99999  999     Census Tract 9604   
4    50001960500     140     50       1  99999  999     Census Tract 9605   
5    50001960700     140     50       1  99999  999     Census Tract 9607   
6    50001960800     140     50       1  99999  999     Census Tract 9608   
7    50001960900     140     50       1  99999  999     Census Tract 9609   
8    50001961000     140     50       1  99999  999     Census Tract 9610   
9    50003970300     140     50       3  13540  999     Census Tract 9703   
10   50003970600     140     50       3  13540  999     Census Tract 9706   
11   50003970900     140     50       3  13540  999     Census Tract 9709   